# CSCI 3155: Assignment 9

Topics: 
- Type Checking
- Basics of Objects
- Generics

Readings: Notes posted on moodle.  Chapters from Odersky

- Chapter 4 (revise)
- Chapter 10
- Chapter 11 (take note of Any, AnyVal, Null, Nothing)
- Chapter 12 
- Chapter 19

__Name__: WRITE YOUR NAME HERE

In [ ]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

## Problem 1 (25 points)

In this problem, we will extend our type system for handling references.  Here is a stripped down version of Lettuce that involves references.

$$\begin{array}{rcl}
\mathbf{Expr} & \rightarrow & Const(\mathbf{Double}) \\
& | & Ident(\mathbf{Identifier})\\
& | & Plus(\mathbf{Expr}, \mathbf{Expr})\\
& | & Div(\mathbf{Expr}, \mathbf{Expr}) \\
& | & Geq(\mathbf{Expr}, \mathbf{Expr}) \\
& | & And(\mathbf{Expr}, \mathbf{Expr}) \\
& | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr})\\
& | & Let(\mathbf{Identifier}, \color{red}{\mathbf{Type}}, \mathbf{Expr}, \mathbf{Expr})\\
& | & FunDef(\mathbf{Identifier}, \color{red}{\mathbf{Type}}, \mathbf{Expr}) \\
& | & FunCall(\mathbf{Expr}, \mathbf{Expr}) \\
& | & \color{blue}{NewRef(\mathbf{Expr})} \\
& | & \color{blue}{DeRef(\mathbf{Expr})} \\
& | & \color{blue}{AssignRef(\mathbf{Expr}, \mathbf{Expr})} \\
\end{array}$$

We have a type system 
$$\begin{array}{rcl}
\mathbf{Type} & \rightarrow & NumType & \leftarrow \text{written as } \ num \\
& | & BoolType & \leftarrow \text{written as } \ bool \\
& | & FunType(\mathbf{Type} , \mathbf{Type} ) & \leftarrow \text{written as } \ t_1 \Rightarrow t_2 \\
& | & \color{red}{RefType(\mathbf{Type})} & \leftarrow \text{written as } \ ref(t_1) \\
\end{array}$$


### A (5 points)
Copy the program below to the next cell and add the missing type annotations labeled type1, type2, and type3 to the the program.


~~~
let x: type1 = 25 in 
    let y : type2 = NewRef( - x ) in 
     let f : type3 = NewRef( function (z: ref(num)) DeRef(z) - x >= 45 ) in 
          let z : ref(bool) = DeRef(f)(y)
~~~

## Solution
* type1 = num --or-- NumType
* type2 = ref(num) --or-- Reftype(NumType)
* type3 = ref(ref(num) => bool) -- or -- Reftype(FunType( RefType(NumType), BoolType))          

### B (10 points): Let us write some inference rules for working with ref types.
$$\newcommand\typeOf{\mathbf{typeOf}}$$
$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}\;(\text{#3}) }$$

Recall from notes on "Types and Type Checking" that $\typeOf(\texttt{e}, \alpha)$ is the type of an 
expression $\texttt{e}$ under type environment $\alpha$. The type environment maps identifiers in the current scope to  their annotated types.


Let us write a rule for NewRef.

$$\semRule{ \typeOf(\texttt{e}, \alpha) = t,\; t \not= \mathbf{typeerror} }{\typeOf(\texttt{NewRef(e)}, \alpha) = ref(t) }{newref-ok}$$

It says that if $\texttt{e}$ receives type $t$ under type environment $\alpha$ and it is not a type error, then $\texttt{NewRef(e)}$ must receive the type $ref(t)$ under $\alpha$.


(i) Complete the missing terms for the rule for `DeRef` OK rule.
$$\semRule{\typeOf(\texttt{e}, \alpha) = ref(t)}{\typeOf(\texttt{DeRef(e)}, \alpha) = \color{red}{???_1}}{deref-ok}$$

(ii) Complete the missing terms for the rule for `DeRef` error.
$$\semRule{\typeOf(\texttt{e}, \alpha) = t,\ t \not= \color{red}{???_2} }{\typeOf(\texttt{DeRef(e)}, \alpha) = \mathbf{typeerror}}{deref-nok}$$

(iii) Complete the missing terms for `AssignRef` OK rule.
$$\semRule{\typeOf(\texttt{e1}, \alpha) = \color{red}{???_3}, \typeOf(\texttt{e2}, \alpha) = t}{\typeOf(\texttt{AssignRef(e1, e2)}, \alpha) =   \color{red}{???_4}}{assignref-ok}$$


Write your answers in the next cell.

## Solution



$\color{red}{???_1} $ = t

$\color{red}{???_2} $ = ref(t_2)

$\color{red}{???_3} $ = ref(t)

$\color{red}{???_4} $ = t

### C (10 points): Write the type checker with references by filling in the missing code.

In [ ]:
sealed trait Type
case object NumType extends Type
case object BoolType extends Type
case class FunType(t1: Type, t2: Type) extends Type
// TODO: Write a new case class for RefType
//BEGIN SOLUTION
case class RefType(t: Type) extends Type
//END SOLUTION

sealed trait Program
sealed trait Expr

case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Geq(e1: Expr, e2: Expr) extends Expr
case class IfThenElse(e1: Expr, e2: Expr, e3: Expr) extends Expr
case class Let(x: String, xType: Type, e1: Expr, e2: Expr) extends Expr
case class FunDef(id: String, idType: Type, e: Expr) extends Expr
case class FunCall(calledFun: Expr, argExpr: Expr) extends Expr
case class NewRef(e: Expr) extends Expr
case class DeRef(e: Expr) extends Expr
case class AssignRef(e1: Expr, e2: Expr) extends Expr
case class TopLevel(e: Expr) extends Program

def typeEquals(t1: Type, t2: Type): Boolean = t1 == t2
case class TypeErrorException(s: String) extends Exception


In [ ]:
def typeOf(e: Expr, alpha: Map[String, Type]): Type = {
    def checkType(opName: String, e1: Expr, t1: Type, e2: Expr, t2: Type, resType: Type): Type = {
        val t1hat = typeOf(e1, alpha)
        if (! typeEquals(t1hat, t1)){
            throw new TypeErrorException(s"Type mismatch in arithmetic/comparison/bool op $opName, Expected type $t1, obtained $t1hat")
        }
        
        val t2hat = typeOf(e2, alpha)
        if (! typeEquals(t2hat, t2)){
            throw new TypeErrorException(s"Type mismatch in arithmetic/comparison/bool op $opName, Expected type $t2, obtained $t2hat")
        }
        
        resType
    }
    
    e match {
        case Const(f) => NumType
        case Ident(s) => {if (alpha contains s)
                             alpha(s)
                          else 
                             throw TypeErrorException(s"Unknown identifier $s")}
        case Plus(e1, e2) =>  checkType("Plus", e1,  NumType, e2, NumType, NumType)
        case Minus(e1, e2) => checkType("Minus",e1,  NumType, e2, NumType, NumType)
        case Geq(e1, e2) => checkType("Geq", e1,  NumType, e2, NumType, BoolType)
        case IfThenElse(e, e1, e2) => {
            val t = typeOf(e, alpha)
            if (t == BoolType){
                val t1 = typeOf(e1, alpha)
                val t2 = typeOf(e2, alpha)
                if (typeEquals(t1, t2))
                    t1
                else 
                    throw TypeErrorException(s"If then else returns unequal types $t1 and $t2")
            } else {
                throw TypeErrorException(s"If then else condition expression not boolean $t")
            }
        }

        case Let(x, t, e1, e2) => {
            val t1 = typeOf(e1, alpha)
            if (typeEquals(t1, t)){
                val newAlpha = alpha + (x -> t)
                typeOf(e2, newAlpha)
            } else {
                throw TypeErrorException(s"Let binding has type $t whereas it is bound to expression of type $t1")
            }
        }

        case FunDef(x, t1, e) => {
            val newAlpha = alpha + (x -> t1)
            val t2 = typeOf(e, newAlpha)
            FunType(t1, t2)
        }

        case FunCall(e1, e2) => {
            val ftype = typeOf(e1, alpha)
            ftype match {
                case FunType(t1, t2) => {
                    val argType = typeOf(e2, alpha)
                    if (typeEquals(argType, t1)){
                        t2
                    } else {
                        throw TypeErrorException(s"Call to function with incompatible argument type. Expected $t1, obtained $argType")
                    }
                }
                case _ => { throw TypeErrorException(s"Call to function but with a non function type $ftype")}

            }
        }

        case NewRef(e) => {
            //BEGIN SOLUTION
            val t = typeOf(e, alpha)
            RefType(t)
            //END SOLUTION
        }
        
        case AssignRef(e1, e2) => {
            //BEGIN SOLUTION
            val t1 = typeOf(e1, alpha)
            val t2 = typeOf(e2, alpha)
            t1 match {
                case RefType(tHat) => if (typeEquals(tHat, t2))
                                         t2
                                      else 
                                         throw new TypeErrorException(s"Assignment of reference with type $tHat incompatible with value of type $t2")
                case _ => throw new TypeErrorException(s"Assignment made to non reference type $t1")
            }
            //END SOLUTION
        }
        
        case DeRef(e) => {
            //BEGIN SOLUTION
            val t = typeOf(e, alpha)
            t match {
                case RefType(tHat) => tHat
                case _ => throw new TypeErrorException(s"Deref of non reference type $t")
            }
            //END SOLUTION
        }
        
    }
}

def typeOfProgram(p: Program) = p match {
    case TopLevel(e) => {
            val t = typeOf(e, Map())
            println(s"Program type computed successfully as $t")
            t
    }
}

In [ ]:
//BEGIN TEST

/* 
let x : ref(num) = NewRef(10 ) in 
   let dummy: num = AssignRef(x, 30) in 
       DeRef(x)
       */

val p1 = Let("x", RefType(NumType), NewRef(Const(10)), Let("dummy", NumType, AssignRef(Ident("x"), Const(30) ), DeRef(Ident("x"))) )
val t1 = typeOfProgram(TopLevel(p1))
assert(t1 == NumType, "Test 1 failed: answer should be NumType")
passed(2)
//END TEST

In [ ]:
//BEGIN TEST
/* 
let x : ref(num) = NewRef(function(z: num) z + 10) in 
   let dummy: num = AssignRef(x, 30) in 
       DeRef(x)
       */
val fdef = FunDef("z", NumType, Plus(Ident("z"), Const(10)))
val p2 = Let("x", RefType(NumType), NewRef(fdef), Let("dummy", NumType, AssignRef(Ident("x"), Const(30) ), DeRef(Ident("x"))) )
val t2 = try {
   typeOfProgram(TopLevel(p2))
   assert(false, "The program should not receive a type")
} catch {
    case TypeErrorException(msg) => s"OK -- caught a type error exception: $msg"
    case e => print(e); assert(false, "Please throw TypeErrorException(message) when a type failure occurs")
}
passed(2)
//END TEST

In [ ]:
//BEGIN TEST
/* 
let x : ref(num => num) = NewRef(function(z: num) z + 10) in 
   let dummy: num = AssignRef(x, 30) in 
       DeRef(x)
       */
val fdef = FunDef("z", NumType, Plus(Ident("z"), Const(10)))
val p3 = Let("x", RefType(FunType(NumType, NumType)), NewRef(fdef), Let("dummy", NumType, AssignRef(Ident("x"), Const(30) ), DeRef(Ident("x"))) )
val t3 = try {
   typeOfProgram(TopLevel(p3))
   assert(false, "The program should not receive a type")
} catch {
    case TypeErrorException(msg) => s"OK -- caught a type error exception: $msg"
    case e => print(e); assert(false, "Please throw TypeErrorException(message) when a type failure occurs")
}
passed(2)
//END TEST

In [ ]:
//BEGIN TEST
/* 
let x : ref(num => num) = NewRef(function(z: num) z + 10) in 
   let dummy: num = AssignRef(NewRef(35), 30) in 
       DeRef(x)
       */
val fdef = FunDef("z", NumType, Plus(Ident("z"), Const(10)))
val p4 = Let("x", RefType(FunType(NumType, NumType)), NewRef(fdef), Let("dummy", NumType, AssignRef(NewRef(Const(35)), Const(30) ), DeRef(Ident("x"))) )
val t4 =  typeOfProgram(TopLevel(p4))
assert(t4 == FunType(NumType, NumType), "Test failed")
passed(2)
//END TEST

In [ ]:
//BEGIN TEST

val y = Ident("y")
val x = Ident("x")
val z = Ident("z")
val t1 = RefType(NumType)
val t2 = FunType(t1, NumType)
val t3 = RefType(t2)

val v1 = FunCall(DeRef(y), x)
val v2 = Let("z", RefType(BoolType), NewRef(Geq(DeRef(x), Const(25))), v1)
val fdef = FunDef("z", RefType(NumType), Plus(DeRef(z), DeRef(x)))
val v3 = Let("y", t3, NewRef(fdef), v2)
val v4 = Let("x", t1, NewRef(Const(10)), v3)

val p5 = TopLevel(v4)
assert(typeOfProgram(p5) == NumType, "Test failed")
passed(2)

//END TEST

## Problem 2 (20 points): Objects in Scala

### A (12 points)
Consider the class below for an employee record in an organization that has three fields `name`, `eid` (employee id) and `salary`.

1) As given, scala will complain that class employee cannot extend the abstract class People. Add the missing methods to enable the class Employee to properly extend People

2) Write a factory pattern that takes in a formatted string of the form "John Doe,29585,33110" consisting of a string and two numbers, to extract the name and salaries for each employee. You may want to use `scala.util.matching.Regex` [https://www.scala-lang.org/api/2.9.2/scala/util/matching/Regex.html]
If the input fails to match the pattern, an `IllegalArgumentException` must be thrown

Companion objects are covered in scala book Chapter 4.3. Also lookup here: https://alvinalexander.com/scala/factory-pattern-in-scala-design-patterns

In [ ]:
import scala.util.matching.Regex

abstract class People {
    /* I need the ability to convert every person to a string */
    def toString: String
    /* I need the ability to check if two people are the same */
    def equals (p: People): Boolean
}

/* 
  Note that for an employee to be equal to an object of type People, that object
  must be an employee with matching eid number 
*/

class Employee( val name: String, val eid: Int, var salary: Int) extends People {
    //BEGIN SOLUTION
    override def toString: String = {
       (s"Employee Name: $name, EID: $eid, Salary: $salary")
    }
    
    override def equals(p: People): Boolean = {
        if (p.isInstanceOf[Employee]){
            val q = p.asInstanceOf[Employee]
            q.eid == this.eid
        } else 
            false
        
    }
    //END SOLUTION
}

object Employee {
    def apply(formattedInput: String): Employee = {
        /* 
        Input format must be 
        (full name with possible spaces)[optional whitespaces],[optional whitespaces](employee ID number)[optional whitespaces],[optional whitespaces](employee salary)
        */
        //BEGIN SOLUTION
        val rex = raw"([A-Za-z ]*)\s*,\s*(\d+)\s*,\s*(\d+)".r
        formattedInput match {
            case rex(name, i1, i2) => { new Employee(name.trim, i1.toInt, i2.toInt)}
            case _ => throw new IllegalArgumentException(s"Badly formatted input $formattedInput")
        }
        //END SOLUTION
    }
}

In [ ]:
//BEGIN TEST
val e1 = Employee("Jonathan Doe, 12893, 39813")
println(e1.toString)
assert(e1.name == "Jonathan Doe", "Failed")
assert(e1.eid == 12893, "Failed")
assert(e1.salary == 39813, "Failed")
assert(e1.equals(new Employee("whoknows", 12893, 39813)))
passed(4)
//END TEST

In [ ]:
//BEGIN TEST
try {
    val e2 = Employee("Badly Formatted String, 29812, $3102")
    assert(false, "Failed, the string is badly formatted but your code is OK with it.")
} catch {
    case e => println(s"Expected behavior seen")
}
passed(4)
//END TEST

In [ ]:
//BEGIN TEST
val e1 = Employee("Janet Maria Catherine Doe    , 12894   , 121331")
println(e1.toString)
assert(e1.name == "Janet Maria Catherine Doe", "Failed")
assert(e1.eid == 12894, "Failed")
assert(e1.salary == 121331, "Failed")
assert(e1.equals(new Employee("random", 12894, 121332)))
passed(4)
//END TEST

### B (8 points)
We wish to add a new class of people to our system. These are "Contractor" with the information associated should involve their name (string), contract ID (Integer) and contract supervisor (Employee).

Write a new class named `Contractor` whose constructor should take in the name of the person (String), their ID number (Integer) and Supervisor (Employee) and extend from People. 

A contractor object is equal to an object p of type People, only if p is actually an instance of contractor, and if their names are the same and they have the same contract ID number.

In [ ]:
//BEGIN SOLUTION
class Contractor (val name: String, val idNumber: Int, val e: Employee) extends People {
    override def toString: String = s"Contractor $name, id: $idNumber, supervisor: $e"
    override def equals(p: People) = {
        if (p.isInstanceOf[Contractor]){
            val q = p.asInstanceOf[Contractor]
            (q.name == name) && (q.idNumber == idNumber)
        } else false
    }
}
//END SOLUTION

In [ ]:
//BEGIN TEST
val e1 = Employee("Jonathan Doe, 12893, 39813")
val c1 = new Contractor("John Contract Worker", 12093, e1)
assert( !c1.equals(e1), "Failed")
assert( c1.equals(c1), "Failed")
passed(4)
//END TEST

In [ ]:
//BEGIN TEST
val e1 = Employee("Jonathan Doe, 12893, 39813")
val e2 = Employee("Jane Doe, 12894, 43814")
val c1 = new Contractor("John Contract Worker", 12093, e2)
val c2 = new Contractor("John Contract Worker", 12093, e1)
assert( c1.equals(c2), "Failed")
passed(4)
//END TEST

## Problem 3 (10 points): Traits in Scala

For this problem, we have defined two traits: `NumberOfLegs` that helps us define how many legs a given animal has and `WarmBlooded` that applies to warm blooded animals.

In [ ]:
abstract class Animal 

trait NumberOfLegs {
    val nLegs: Int
    def getNumberOfLegs: Int = nLegs
}

trait WarmBlooded extends Animal {
    val bodyTempMaintained: Double
    def getBodyTemp: Double = bodyTempMaintained 
}

### A (5 points)

Define a class `Human` that inherits from `Animal` and mixes in the traits `NumberOfLegs` with `nLegs = 2` and `WarmBlooded` with `bodyTempMaintained = 98`. The class `Human` must take in a parameter called `name` of type `String` and implement a `getName` method without any parameters.

In [ ]:
//BEGIN SOLUTION
class Human(val name: String) extends Animal with NumberOfLegs with WarmBlooded {
    override val nLegs:Int = 2
    val bodyTempMaintained = 98.0
    def getName: String = name
}
//END SOLUTION

In [ ]:
//BEGIN TEST
val t1 = new Human("Jane Smith")
assert(t1.getNumberOfLegs == 2, "Your human does not have two legs")
assert(t1.bodyTempMaintained == 98.0, "Your human does not maintain a body temp of 98")
assert(t1.getBodyTemp == 98.0, "Your human's getBodyTemp Function is not working")
assert(t1.getName == "Jane Smith", "Your human's name is not setting correctly")
passed(5)
//END TEST

### B (5 points)
Define a class named `Table` that mixes in the trait `NumberOfLegs` with `nLegs = 4`. 

_Note:_ `Table` cannot mixin the trait `WarmBlooded` (make sure you understand why that is as a quick check of concepts).

In [ ]:
//BEGIN SOLUTION
class Table extends NumberOfLegs {
    val nLegs = 4
}
//END SOLUTION

In [ ]:
//BEGIN TEST
val tbl = new Table()
assert(tbl.getNumberOfLegs == 4, "A Table must have four legs")
passed(5)
//END TEST